In [17]:
import numpy as np
import pandas as pd
import sys
import time
from graphics import *

In [27]:
class Card:
    
    def __init__(self, rank, suit, score = 0):
        self.rank = rank
        self.suit = suit
        self.score = score
        
        
        # initializing card art:
        def center(width, suit):
            return " " * 2 + self.getSuit() + " " * 2
        
        self.art = []
        width = 14

        self.art.append("*" * (width + 1))
        for i in range(width):
            currLine = "*"
            if i == 1:
                currLine += ("  " + str(self.getRank()) + " " * 7 + str(self.getRank()) + "  ")
            elif i == 7:
                currLine += center(width, self.getSuit())
            elif i == 12:
                currLine += "  " + str(self.getRank()) + " " * 7 + str(self.getRank()) + "  "
            else:
                currLine += " " * 8
            while len(currLine) < width:
                currLine += " "
            currLine += "*"
            self.art.append(currLine)
        self.art.append("*" * (width + 1))
    
    def getSuit(self):
        return self.suit
    
    def getRank(self):
        return self.rank
    
    def getScore(self):
        return self.score
    
#     def __str__(self): # Text based toString
#         if self.rank == 1:
#             return "A of " + self.suit
#         elif self.rank < 11:
#             return str(self.rank) + " of " + self.suit
#         elif self.rank == 11:
#             return "J of " + self.suit
#         elif self.rank == 12:
#             return "Q of " + self.suit
#         elif self.rank == 13:
#             return "K of " + self.suit
#         return str(self.rank) + " of " + self.suit

    def __str__(self):
        return "\n".join(self.art)

In [28]:
class Deck:
    
    def __init__(self):
        self.cards = []
        self.count = 0 #consider keeping track of count and doing a test to see how much of a difference it makes
                       #look into varying the shuffling frequency    
        self.shuffle()
        
    
    def shuffle(self):
        ranks = [1,2,3,4,5,6,7,8,9,10,11,12,13]
        suits = ["Hearts", "Diamonds", "Spades", "Clubs"]
    
        for rank in ranks:
            for suit in suits:
                score = rank
                if score > 10:
                    score = 10
                if score == 1:
                    score = 11
                    
                self.cards.append(Card(rank, suit, score))
                
        print("*** SHUFFLING DECK ***")
        self.count = 0
        self.cards = list(pd.Series(self.cards).sample(replace=False, frac=1))
    
    def deal(self):
        card = self.cards.pop()
        
        if card.score >= 2 and card.score <= 6:
            self.increaseCount()
        elif card.score >= 10:
            self.decreaseCount()
        else:
            pass # do nothing for 7,8,9
            
        if self.getNumCards() == 0:
            self.shuffle() 
        return card
    
    def getNumCards(self):
        return len(self.cards)
    
    def increaseCount(self):
        self.count += 1
        
    def decreaseCount(self):
        self.count -= 1
        
    def getCount(self):
        return self.count
    
    def printDeck(self):
        """
        Only for debugging
        """
        for card in self.cards:
            print(card.rank, "of", card.suit)

In [29]:
class Hand:
    
    def __init__(self):
        self.cards = []
        self.score = 0
        self.busted = False
        self.standing = False
        
        
    def hit(self, card):
        self.cards.append(card)
        self.score += card.score
        if card.rank == 1 and self.score > 21:
            self.score -= 10
        if self.score > 21:
            self.busted = True
    
    def hasBlackjack(self):
        if len(self.cards) == 2 and self.score == 21:
            return True
        return False
    
    def isBusted(self):
        return self.busted
    
    def getCards(self):
        return self.cards
    
    def __str__(self):
        return " | ".join([str(card) for card in self.cards]) + " | [{}]".format(self.score)

In [30]:
class Player(Hand):
    
    def __init__(self, card = None):
        super().__init__()
        self.canDoubleDown = True
        self.canSplit = False
        if card is not None:
            self.cards.append(card)
            self.score += card.score
        
    def hit(self, card):
        self.cards.append(card)
        self.score += card.score
        
        if card.rank == 1 and self.score > 21:
            self.score -= 10
        if self.score > 21:
            self.busted = True
        if self.canSplit:
            self.canSplit = False
        if len(self.cards) == 2 and self.cards[0].getScore() == self.cards[1].getScore():
            self.canSplit = True
        if len(self.cards) == 3:
            self.canDoubleDown = False
            
    def executeTurn(self, game):
        first = True
        while True:
            if not first:
                game.printBoard(hidden = True)
            first = False
            
            if self.standing or self.busted:
                return
            
            if self.canSplit and self.canDoubleDown:
                move = (input("*** Would you like to hit (H), stand (S), double down (D), or split (SP)? *** ").lower())
                if move == "h":
                    self.hit(game.deck.deal())
                    continue
                elif move == "s":
                    self.standing = True
                    continue
                elif move == "d":
                    self.doubleDown(game)
                    self.canDoubleDown = False
                    continue
                elif move == "sp":
                    self.split(game)
                    continue
                else:
                    print("*** Please enter a valid move ***")
                    continue      
            elif self.canSplit:
                move = (input("*** Would you like to hit (H), stand (S), or split (SP)? *** ").lower())
                if move == "h":
                    self.hit(game.deck.deal())
                    continue
                elif move == "s":
                    self.standing = True
                    continue
                elif move == "sp":
                    self.split(game)
                    continue
                else:
                    print("*** Please enter a valid move ***")
                    continue  
            elif self.canDoubleDown: # most common starter
                move = (input("*** Would you like to hit (H), stand (S), or double down (D)? *** ").lower())
                if move == "h":
                    self.hit(game.deck.deal())
                    continue
                elif move == "s":
                    self.standing = True
                    continue
                elif move == "d":
                    self.doubleDown(game)
                    continue
                else:
                    print("*** Please enter a valid move ***")
                    continue
            else:
                move = (input("*** Would you like to hit (H) or stand (S)? *** ").lower())
                if move == "h":
                    self.hit(game.deck.deal())
                    continue
                elif move == "s":
                    self.standing = True
                    continue
                else:
                    print("*** Please enter a valid move ***")
                    continue
    
      
   
    def doubleDown(self, game):
        game.setBet(game.getBet() * 2)
        self.hit(game.deck.deal())
        self.standing = True
        return
        
    def split(self, game):
        # todo, use overloaded constructor
        g1 = BlackJackGame(Player(), game.getDealer(), game.getChips(), game.getBet(), game.getShowCount())
        g2 = BlackJackGame(Player(), game.getDealer(), game.getChips(), game.getBet(), game.getShowCount())
        return g1.playHand(game.bet) + g2.playHand(game.bet)
        


In [31]:
class Dealer(Hand):
    def __init__(self):
        super().__init__()
        
    def executeTurn(self, game):
        game.printBoard()
        time.sleep(1.5)
        while not self.standing and not self.busted:
            if self.score < 17:
                self.hit(game.deck.deal())
                game.printBoard()
                time.sleep(1.5)
            else:
                self.standing = True 
                
    def __str__(self):
        if len(self.cards) == 2:
            return str(self.cards[0]) + " | ????? | [{}]".format(self.score - self.cards[1].score)
        else:
            return super().__str__()

In [32]:
class BlackJackGame:
    
    def __init__(self, player, dealer, chips, bet, showCount):
        self.player = player
        self.dealer = dealer
        self.deck = Deck()
        
        self.showCount = showCount
        self.chips = chips
        self.bet = 0
        
    def playHand(self, bet, doubledDown = False):
        """
        Returns the resulting change in player's chip count
        """
        if not doubledDown:
            self.player = Player()
            self.dealer = Dealer()
        self.bet = bet
        
        self.player.hit(self.deck.deal())
        self.dealer.hit(self.deck.deal())
        self.player.hit(self.deck.deal())
        self.dealer.hit(self.deck.deal())
        
        self.printBoard(hidden = True)
        

        if (self.player.hasBlackjack() and self.dealer.hasBlackjack()):
            print("*** Push! ***")
            return 0

        if (self.player.hasBlackjack()):
            print("*** You got blackjack! ***")
            return bet*1.5

        if (self.dealer.hasBlackjack()):
            self.printBoard(hidden = False)
            print("*** The dealer got blackjack, you lose ***")
            return -1*bet
        
        self.player.executeTurn(self)
        if self.player.isBusted():
            print("*** You lose! You went over 21 ***")
            return -1*bet
        
        self.dealer.executeTurn(self)
        if self.dealer.isBusted():
            print("*** You win! The dealer went over 21 ***")
            return bet
        
        if self.dealer.score == self.player.score:
            print("*** Push! ***")
            return 0
        
        if self.dealer.score > self.player.score:
            print("*** You lose! The dealer had a higher score ***")
            return -1*bet
        
        if self.player.score > self.dealer.score:
            print("*** You win! You had a higher score ***")
            return bet

        print("returning 0 for no reason, todo")
        return 0
    
    def setChips(self, chips):
        self.chips = chips
        
    def getChips(self):
        return self.chips
    
    def getDealer(self):
        return self.dealer
    
    def getDeck(self):
        return self.deck
    
    def setBet(self, bet):
        self.bet = bet
        
    def getBet(self):
        return self.bet
    
    def setShowCount(self, val):
        self.showCount = val
        
    def getShowCount(self):
        return self.showCount
        
    def printBoard(self, hidden = False):
        print("*"*60)
        print("*** CURRENT CHIP COUNT: ${} *** CURRENT BET: ${} ***".format(self.chips, self.bet))
        if hidden:
            print("*** DEALERS HAND: {} ***".format(self.dealer))
        else:
            print("*** DEALERS HAND: {} ***".format(super(type(self.dealer), self.dealer).__str__()))
        print("*** YOUR HAND: {} ***".format(self.player))
        print("*"*60)
        # this is gona be fun, dont forget self.showCount # is it now?

In [33]:
class CardCountingPractice:
    def __init__(self, speed, cpd, deck = Deck()):
        self.count = 0
        self.speed = speed # 1, 2, 3
        self.cpd = cpd # 1, 2, 3 # cpd is cards per deal
        self.deck = deck
        
    def start(self):
        pass
    
    def playCardCountingPractice(self):
        while True:
            self.speed = int(input("*** Pick your speed difficulty: easy (1), medium (2), or hard (3) *** "))
            if speed == 1:
                self.sleep = 3
                break
            elif self.speed == 2: 
                self.sleep = 2
                break
            elif self.speed == 3:
                self.sleep = 1
                break
            else:
                print("*** Please enter a valid speed *** ")

        while True:
            self.cpd = int(input("*** Pick your cards per deal difficulty: 1, 2, or 3 *** "))
            if self.cpd == 1 or self.cpd == 2 or self.cpd == 3:
                break
            else:
                print("*** Please enter a valid cards per deal ***")

        self.numDeals = 5 + np.random.randint(5) # 5 thru 9
        playAgain = True
        while playAgain:
            if (self.numDeals * self.cpd) > self.deck.getNumCards():
                print("*** End of deck ***")
                self.deck.shuffle()
                continue
            else:
                for i in range(self.numDeals):
                    cards = []
                    time.sleep(self.sleep)
                    print("***")
                    for j in range(self.cpd):
                        card = self.deck.deal()
                        print(card)
                        cards.append(card)

            print("***")
            self.test(self.deck.getCount())


            while True:
                resp = input("*** Would you like to play again? (Y/N) ").lower()
                if resp == "y":
                    break
                elif resp == "n":
                    print("*** Thanks for playing! ***")
                    playAgain = False
                    break
                else:
                    print("*** Please enter a valid response ***")


    def test(self, count):
        while True:
            try:
                start = time.time()
                resp = int(input("*** What is the current count? *** "))
                stop = time.time()
                delta = round(stop-start, 2)
                if resp == count:
                    print("*** Correct! ***")
                    print("*** Time elapsed: {} seconds".format(delta))
                else:
                    print("*** Wrong! The correct count is {}".format(count))
                    print("*** Time elapsed: {} seconds".format(delta))
                break
            except ValueError:
                print("*** Please enter a valid count ***")

*** SHUFFLING DECK ***


In [34]:
def playBlackJack():
    while True:
        try:
            chips = int(input("*** How many chips would you like to start with? *** "))
            if chips <= 0:
                raise ValueError()
            break
        except ValueError:
            print("*** Enter a valid chip amount ***")

    first = True
    while True:
        try:
            print("*"*60)
            if first:
                print("*** CURRENT CHIP COUNT: ${} ***".format(chips))
            else:
                print("*** CURRENT CHIP COUNT: ${} ***".format(game.chips))
            bet = int(input("*** How much would you like to bet? *** "))
            if bet < 0 or bet > chips:
                raise ValueError()
            if first:
                game = BlackJackGame(Player(), Dealer(), chips, bet, showCount = True)
                first = False
            game.setChips(game.getChips() + game.playHand(bet))
            if game.getChips() <= 0:
                break
            print("\n\n")
        except ValueError:
            print("*** Enter a valid chip amount ***")

    print("*** You ran out of chips ***")

In [35]:
def main():
    while True:
        decision = input("*** Enter 1 for BlackJack ***\n*** Enter 2 for CardCountingPractice ***\n*** Enter 3 to exit ***\n*** What would you like to play? *** ")
        if decision == '1':
            playBlackJack()
            break
        elif decision == '2':
            playCardCountingPractice()
            break
        elif decision == '3':
            sys.exit(0)
        else:
            continue
        
main()

*** Enter 1 for BlackJack ***
*** Enter 2 for CardCountingPractice ***
*** Enter 3 to exit ***
*** What would you like to play? *** 3


SystemExit: 0

/Users/kylenero/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [36]:
card = Card(5, "Clubs")
print(card)

***************
*             *
*  5       5  *
*             *
*             *
*             *
*             *
*             *
*  Clubs      *
*             *
*             *
*             *
*             *
*  5       5  *
*             *
***************


In [53]:
"*" + "Clubs".ljust(10)

'*Clubs     '

In [54]:
len("***************")

15

In [39]:
def center(width, suit):
    # n spaces, word, n spaces # 13 total without the outer stars
    # if len(suit) is 5, 
    usable = width - 2
    


In [40]:
art = []
width = 14

art.append("*" * (width + 1))
for i in range(width):
    currLine = "*"
    if i == 1:
        currLine += ("  " + str(card.getRank()) + " " * 7 + str(card.getRank()) + "  ")
    elif i == 7:
        currLine += center(width, card.suit)
    elif i == 12:
        currLine += "  " + str(card.getRank()) + " " * 7 + str(card.getRank()) + "  "
    else:
        currLine += " " * 8
    while len(currLine) < width:
        currLine += " "
    currLine += "*"
    art.append(currLine)
art.append("*" * (width + 1))
"\n".join([str(line) for line in art])

'***************\n*             *\n*  5       5  *\n*             *\n*             *\n*             *\n*             *\n*             *\n*  Clubs      *\n*             *\n*             *\n*             *\n*             *\n*  5       5  *\n*             *\n***************'